In [ ]:
params_lgbm = {'num_leaves': 34, 'min_child_samples': 16, 
          'max_depth': 8, 'learning_rate': 0.012, 
          'min_sum_hessian_in_leaf': 1e-4,
          'objective': 'regression_l1', 'feature_fraction': 0.9574152630927155,
          'n_jobs':-1, 'num_iterations':5000
          }
def objective(trial):
    

    params_enet = {
        'alpha': trial.suggest_float('alpha', 0.0001, 1.0),
        'l1_ratio': trial.suggest_float('l1_ratio', 0.0, 1.0)
    }

    N_TRAIN = 0.85
    # Определение моделей
    lgbm_model = LGBMRegressor(**params_lgbm)


    # Определение мета-модели (можно использовать любую модель; в этом примере используется ElasticNet)
    meta_model = ElasticNet(alpha=params_enet['alpha'],l1_ratio=params_enet['l1_ratio'])

    # Создание стекинг-регрессора
    stacking = StackingRegressor(estimators=[('lgbm', lgbm_model)],
                                                final_estimator=meta_model, 
                                               
                                                n_jobs= -1
                                                )
   
    
    X_train, X_test = feat_lgbm_train[:int(len(feat_lgbm_train)*N_TRAIN)], feat_lgbm_train[int(len(feat_lgbm_train)*N_TRAIN):]
    y_train, y_test = target_all_train[:int(len(feat_lgbm_train)*N_TRAIN)], target_all_train[int(len(feat_lgbm_train)*N_TRAIN):]


    stacking.fit(X_train, y_train)
    y_pred = stacking.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    rmse = r2_score(y_test, y_pred)
    
    
    trial.set_user_attr('MAE', mae)
    trial.set_user_attr('RMSE', rmse)
   

    return mae


In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100, n_jobs = -1, show_progress_bar=True)

In [ ]:
print(study.best_trial.value)

In [ ]:
print(study.best_params)

In [ ]:
params_xgb = {
    'max_depth': [7],  # или другие значения, которые вы хотите проверить
    'n_estimators': [195],  # или другие значения
    'learning_rate': [0.1],  # или другие значения
    'tree_method': ['exact'],  # или другие значения
    'objective': ['reg:squarederror'],  # или другие значения
    'eval_metric': ['rmse'],  # или другие значения
    'gamma': [2],  # или другие значения
    'colsample_bytree': [1],  # или другие значения
    'random_state': [random_state]  # или другие значения
}

def objective(trial):

    params_enet = {
        'alpha': trial.suggest_float('alpha', 0.0001, 1.0),
        'l1_ratio': trial.suggest_float('l1_ratio', 0.0, 1.0)
    }

    N_TRAIN = 0.85
    # Определение моделей
    lgbm_model = LGBMRegressor(**params_lgbm)
    xgb_model = XGBRegressor(**params_xgb)


    # Определение мета-модели (можно использовать любую модель; в этом примере используется ElasticNet)
    meta_model = ElasticNet(alpha=params_enet['alpha'],l1_ratio=params_enet['l1_ratio'])

    # Создание стекинг-регрессора
    stacking = StackingRegressor(estimators=[('lgbm', lgbm_model), ('xgb', xgb_model)],
                                                final_estimator=meta_model, 
                                                n_jobs = -1
                                                )
   
    
    X_train, X_test = feat_lgbm_train[:int(len(feat_lgbm_train)*N_TRAIN)], feat_lgbm_train[int(len(feat_lgbm_train)*N_TRAIN):]
    y_train, y_test = target_all_train[:int(len(feat_lgbm_train)*N_TRAIN)], target_all_train[int(len(feat_lgbm_train)*N_TRAIN):]


    stacking.fit(X_train, y_train)
    y_pred = stacking.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    rmse = r2_score(y_test, y_pred)
    
    
    trial.set_user_attr('MAE', mae)
    trial.set_user_attr('RMSE', rmse)
  
    
    return mae


In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100, n_jobs = -1)

In [ ]:
print(study.best_trial.value)

In [ ]:
print(study.best_params)

In [ ]:

def objective(trial):
   
    params_sgd = {
        'loss': trial.suggest_categorical('loss', ['squared_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive']),
        'penalty': trial.suggest_categorical('penalty', ['l2', 'l1', 'elasticnet']),
        'alpha': trial.suggest_float('alpha', 0.0001, 1.0),
        'l1_ratio': trial.suggest_float('l1_ratio', 0.0, 1.0),
        'fit_intercept': trial.suggest_categorical('fit_intercept', [True, False]),
        'max_iter': trial.suggest_int('max_iter', 1000, 5000),
        'tol': trial.suggest_float('tol', 1e-5, 1e-1)
    }

    N_TRAIN = 0.85
    # Определение моделей
    lgbm_model = LGBMRegressor(**params_lgbm)
    xgb_model = XGBRegressor(**params_xgb)


    # Определение мета-модели (можно использовать любую модель; в этом примере используется ElasticNet)
    meta_model = SGDRegressor(loss=params_sgd['loss'],
                               penalty=params_sgd['penalty'],
                               alpha=params_sgd['alpha'],
                               l1_ratio=params_sgd['l1_ratio'],
                               fit_intercept=params_sgd['fit_intercept'],
                               max_iter=params_sgd['max_iter'],
                               tol=params_sgd['tol'])

    # Создание стекинг-регрессора
    stacking = StackingRegressor(estimators=[('lgbm', lgbm_model), ('xgb', xgb_model)],
                                                final_estimator=meta_model, 
                                                #cv=5,
                                                n_jobs= -1
                                                )
   
    
    X_train, X_test = feat_lgbm_train[:int(len(feat_lgbm_train)*N_TRAIN)], feat_lgbm_train[int(len(feat_lgbm_train)*N_TRAIN):]
    y_train, y_test = target_all_train[:int(len(feat_lgbm_train)*N_TRAIN)], target_all_train[int(len(feat_lgbm_train)*N_TRAIN):]


    stacking.fit(X_train, y_train)
    y_pred = stacking.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    rmse = r2_score(y_test, y_pred)
    
    
    trial.set_user_attr('MAE', mae)
    trial.set_user_attr('RMSE', rmse)
   
    
    return mae


In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=150, n_jobs = -1)

In [ ]:
print(study.best_trial.value)

In [ ]:
print(study.best_params)

In [ ]:

def objective(trial):
   
    params_xgb = {
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'n_estimators': trial.suggest_int('n_estimators', 10, 200),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5),
        'tree_method': trial.suggest_categorical('tree_method', ['approx', 'auto', 'exact', 'hist']),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.5, 1]),
        'gamma': trial.suggest_int('gamma', 0, 10),
    }


    N_TRAIN = 0.85
    # Определение моделей
    lgbm_model = LGBMRegressor(**params_lgbm)
    xgb_model = XGBRegressor(**params_xgb)


    # Определение мета-модели (можно использовать любую модель; в этом примере используется ElasticNet)
    meta_model = xgb_model(max_depth=params_xgb['max_depth'],
                               n_estimators=params_xgb['n_estimators'],
                               learning_rate=params_xgb['learning_rate'],
                               tree_method=params_xgb['tree_method'],
                               fit_intercept=params_xgb['fit_intercept'],
                               colsample_bytree=params_xgb['colsample_bytree'],
                               gamma=params_xgb['gamma'])

    # Создание стекинг-регрессора
    stacking = StackingRegressor(estimators=[('lgbm', lgbm_model), ('xgb', xgb_model)],
                                                final_estimator=meta_model, 
                                                #cv=5,
                                                passthrough=True,
                                                n_jobs= -1
                                                )
   
    
    X_train, X_test = feat_lgbm_train[:int(len(feat_lgbm_train)*N_TRAIN)], feat_lgbm_train[int(len(feat_lgbm_train)*N_TRAIN):]
    y_train, y_test = target_all_train[:int(len(feat_lgbm_train)*N_TRAIN)], target_all_train[int(len(feat_lgbm_train)*N_TRAIN):]


    stacking.fit(X_train, y_train)
    y_pred = stacking.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    rmse = r2_score(y_test, y_pred)
    
    
    trial.set_user_attr('MAE', mae)
    trial.set_user_attr('RMSE', rmse)
   
    
    return mae


In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100, n_jobs = -1, )

In [ ]:
print(study.best_trial.value)

In [ ]:
print(study.best_params)